In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# 导入mnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [6]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.9379
SVD: 0.9366
NetD, Acc at first 0.938


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9267
SVD: 0.925
NetG, Acc at first 0.9536


In [7]:
# data selected
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 1
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 1

for epoch in range(num_steps):
    if 1==1:
        
        #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
        #radius_list.append(radius)
        #print(radius)

        for i in range(n_dis):
            netD.zero_grad()
            optD.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
            print(errD, errD_EC)
            errD.backward()
            optD.step()
        temp = 0
        for j in range(n_dis2):
            netG.zero_grad()
            optG.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            print(errG, errG_EC)
            temp = errG
            errG = (-1) * errG
            errG.backward()
            optG.step()

            update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
            netV.load_state_dict(update_dic)

        loss_V = temp
        

3.282375793727554
tensor(-49.3840, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.5551, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.5047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-39.3242, device='cuda:0', grad_fn=<AddBackward0>)]
6.488447413775416
tensor(-43.7342, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.8139, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.8866, device='cuda:0', grad_fn=<NegBackward0>), tensor(-36.0337, device='cuda:0', grad_fn=<AddBackward0>)]
99.82392378319837
tensor(-43.5383, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.8139, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.8171, device='cuda:0', grad_fn=<NegBackward0>), tensor(-35.9073, device='cuda:0', grad_fn=<AddBackward0>)]
5.344039752619411
tensor(-64.6911, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-7.1748, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.4032, device='cuda:0', grad_fn=<NegBackward0>), tensor(-52.1131, device='cuda:0', grad_fn=<AddBackw

4.767682891137489
tensor(-98.8089, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.5444, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.2608, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.0037, device='cuda:0', grad_fn=<AddBackward0>)]
4.272326562444623
tensor(-99.5626, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.5444, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1761, device='cuda:0', grad_fn=<NegBackward0>), tensor(-75.8421, device='cuda:0', grad_fn=<AddBackward0>)]
4.316193392957392
tensor(-95.9947, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.8685, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.5227, device='cuda:0', grad_fn=<NegBackward0>), tensor(-73.6035, device='cuda:0', grad_fn=<AddBackward0>)]
4.525807276443729
tensor(-96.4676, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.8685, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.6614, device='cuda:0', grad_fn=<NegBackward0>), tensor(-73.9377, device='cuda:0', grad_fn=<AddB

99.7242071680044
tensor(-100.5563, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.3117, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.1663, device='cuda:0', grad_fn=<NegBackward0>), tensor(-77.0784, device='cuda:0', grad_fn=<AddBackward0>)]
4.491202677262305
tensor(-103.4693, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.4036, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.2797, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.7860, device='cuda:0', grad_fn=<AddBackward0>)]
4.933158802825391
tensor(-105.6795, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.4036, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.5778, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.6981, device='cuda:0', grad_fn=<AddBackward0>)]
5.065740631611357
tensor(-106.9860, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.7324, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7367, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.5169, device='cuda:0', grad_fn=<A

4.418038226272377
tensor(-109.6589, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.3844, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.2789, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.9956, device='cuda:0', grad_fn=<AddBackward0>)]
4.739100687669477
tensor(-109.8723, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.3844, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.6837, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.8042, device='cuda:0', grad_fn=<AddBackward0>)]
4.687787129305302
tensor(-106.9124, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.4830, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.3475, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.0818, device='cuda:0', grad_fn=<AddBackward0>)]
5.014495360960121
tensor(-112.7589, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.4830, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3561, device='cuda:0', grad_fn=<NegBackward0>), tensor(-81.9197, device='cuda:0', grad_fn=<

4.435230890706327
tensor(-111.8292, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-24.8923, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.5430, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.3939, device='cuda:0', grad_fn=<AddBackward0>)]
4.2957258136892404
tensor(-113.8095, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.0625, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4865, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.2605, device='cuda:0', grad_fn=<AddBackward0>)]
4.400713905566676
tensor(-116.2048, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.0625, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8903, device='cuda:0', grad_fn=<NegBackward0>), tensor(-82.2520, device='cuda:0', grad_fn=<AddBackward0>)]
4.5427950659437295
tensor(-115.2502, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.1103, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8293, device='cuda:0', grad_fn=<NegBackward0>), tensor(-81.3106, device='cuda:0', grad_fn

...

4.293308073228067
tensor(-138.5863, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4942, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.7609, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.3313, device='cuda:0', grad_fn=<AddBackward0>)]
4.222276897614415
tensor(-138.7939, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4179, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.8647, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.5113, device='cuda:0', grad_fn=<AddBackward0>)]
4.893823466333419
tensor(-138.7290, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4179, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.5068, device='cuda:0', grad_fn=<NegBackward0>), tensor(-91.8044, device='cuda:0', grad_fn=<AddBackward0>)]
4.278170289309923
tensor(-135.4862, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4744, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.7311, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.2806, device='cuda:0', grad_fn

3.980003467791093
tensor(-137.5259, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.6685, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.7494, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.1081, device='cuda:0', grad_fn=<AddBackward0>)]
4.335802615538196
tensor(-135.8727, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.6685, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.9181, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.2861, device='cuda:0', grad_fn=<AddBackward0>)]
4.346416691843582
tensor(-136.9591, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.6775, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.6995, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.5821, device='cuda:0', grad_fn=<AddBackward0>)]
4.248411348514914
tensor(-133.7729, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.6775, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.0302, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.0653, device='cuda:0', grad_fn=<

4.291981723494245
tensor(-135.8678, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4822, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.6216, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7639, device='cuda:0', grad_fn=<AddBackward0>)]
4.384748713236361
tensor(-134.0940, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.5390, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3525, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.2025, device='cuda:0', grad_fn=<AddBackward0>)]
4.612293855234651
tensor(-139.1730, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.5390, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.1503, device='cuda:0', grad_fn=<NegBackward0>), tensor(-91.4837, device='cuda:0', grad_fn=<AddBackward0>)]
4.421128260624422
tensor(-135.5278, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.4962, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.9306, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.1010, device='cuda:0', grad_fn=

4.7992981175117375
tensor(-135.8030, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.6404, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.7714, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.3912, device='cuda:0', grad_fn=<AddBackward0>)]
4.076325962498677
tensor(-132.6723, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.6404, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.4751, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.5568, device='cuda:0', grad_fn=<AddBackward0>)]
4.74221951368323
tensor(-138.4670, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.7806, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.6201, device='cuda:0', grad_fn=<NegBackward0>), tensor(-91.0662, device='cuda:0', grad_fn=<AddBackward0>)]
4.266035313553013
tensor(-137.2348, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.7806, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.3210, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.1331, device='cuda:0', grad_fn=

4.437386052149896
tensor(-139.1110, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.6673, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.7871, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.6566, device='cuda:0', grad_fn=<AddBackward0>)]
4.485191705169198
tensor(-134.1704, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.5760, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.5618, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.0327, device='cuda:0', grad_fn=<AddBackward0>)]
4.1573171881294115
tensor(-135.4449, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.5760, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.1047, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7643, device='cuda:0', grad_fn=<AddBackward0>)]
4.478673111646959
tensor(-134.9864, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-37.5137, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.6914, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7813, device='cuda:0', grad_fn

In [8]:
radius_list

[3.282375793727554,
 6.488447413775416,
 99.82392378319837,
 5.344039752619411,
 3.8205333949024785,
 5.423148589524126,
 4.6679619660578515,
 56.50843091451363,
 5.391165683725408,
 99.98929168049867,
 4.480202657651448,
 23.071767772777452,
 5.004841595302742,
 4.2301240657062795,
 96.63011105396785,
 4.294531497061677,
 4.745426723573208,
 4.176329749390307,
 4.260300465272114,
 4.445545864164093,
 4.372593014181479,
 4.320363291177695,
 4.304603071534016,
 4.466560587976187,
 4.595281610308871,
 4.228231947313837,
 5.019819347728403,
 4.656077731985817,
 4.283274292114414,
 4.408046746943414,
 5.058305558665332,
 4.632350314433494,
 4.887804185854826,
 4.767682891137489,
 4.272326562444623,
 4.316193392957392,
 4.525807276443729,
 4.493419173583986,
 4.752893029350199,
 5.112037204595058,
 4.486664598898719,
 4.621099912950416,
 4.446457313963167,
 4.640028458585327,
 4.418528072524913,
 5.0349942255894025,
 4.657269004993109,
 4.635664305321241,
 4.521050418599569,
 4.812971615816

In [9]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.74s/it]


----------------------------
Train_z, Test_z
PCA: 0.9358
SVD: 0.923
NetD, Acc at first 0.9628


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9256
SVD: 0.922
NetG, Acc at first 0.9505


In [10]:
torch.save(netD, 'MNIST_NetD-mode1-radius2-makeup.pth')
torch.save(netV, 'MNIST_NetV-mode1-radius2-makeup.pth')
torch.save(netG, 'MNIST_NetG-mode1-radius2-makeup.pth')

In [11]:
#robust testing
r_list = [0,0.1,0.2,0.5,1,2,3,4,6,8,10]
sample_num = 20

In [12]:
# sample 20 times of NetG at final radius, NetD is fixed, length 0.1,0.2,0.5,1,2,4,6,8,10,15,20
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample = []
for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[1])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample.append(count/10000)

print(max(Acc_results_G_sample),min(Acc_results_G_sample),np.mean(Acc_results_G_sample),np.var(Acc_results_G_sample))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.9261
SVD: 0.9149
NetG_fixD, Acc at sample 1 0.9671


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.9277
SVD: 0.9149
NetG_fixD, Acc at sample 2 0.967


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9259
SVD: 0.9155
NetG_fixD, Acc at sample 18 0.9669


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.9261
SVD: 0.9143
NetG_fixD, Acc at sample 19 0.9668


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.79s/it]


----------------------------
Train_z, Test_z
PCA: 0.9266
SVD: 0.9144
NetG_fixD, Acc at sample 20 0.9669
0.9673 0.9663 0.9667549999999998 7.64749999999892e-08


In [13]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample2 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[2])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample2.append(count/10000)
    
print(max(Acc_results_G_sample2),min(Acc_results_G_sample2),np.mean(Acc_results_G_sample2),np.var(Acc_results_G_sample2))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9269000000000001
SVD: 0.9153
NetG_fixD, Acc at sample 1 0.9667


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9271
SVD: 0.9155
NetG_fixD, Acc at sample 2 0.9671


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.927
SVD: 0.9154
NetG_fixD, Acc at sample 18 0.9671


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9271
SVD: 0.916
NetG_fixD, Acc at sample 19 0.9672


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.9268
SVD: 0.9141
NetG_fixD, Acc at sample 20 0.9675
0.9676 0.9659 0.9671299999999998 1.311000000000072e-07


In [14]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample3 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[3])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample3.append(count/10000)
    
print(max(Acc_results_G_sample3),min(Acc_results_G_sample3),np.mean(Acc_results_G_sample3),np.var(Acc_results_G_sample3))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.9288
SVD: 0.9156
NetG_fixD, Acc at sample 1 0.967


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.927
SVD: 0.9137
NetG_fixD, Acc at sample 2 0.967


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9303
SVD: 0.9182
NetG_fixD, Acc at sample 18 0.967


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.929
SVD: 0.9131
NetG_fixD, Acc at sample 19 0.9659


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.9274
SVD: 0.914
NetG_fixD, Acc at sample 20 0.9668
0.9675 0.9657 0.9666549999999999 2.9047500000000176e-07


In [15]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample4 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[4])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample4.append(count/10000)
    
print(max(Acc_results_G_sample4),min(Acc_results_G_sample4),np.mean(Acc_results_G_sample4),np.var(Acc_results_G_sample4))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.931
SVD: 0.9215
NetG_fixD, Acc at sample 1 0.9651


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.9279
SVD: 0.9181
NetG_fixD, Acc at sample 2 0.9649


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9284
SVD: 0.9178999999999999
NetG_fixD, Acc at sample 18 0.9642


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.9309000000000001
SVD: 0.9183
NetG_fixD, Acc at sample 19 0.9658


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.9257
SVD: 0.914
NetG_fixD, Acc at sample 20 0.9649
0.9666 0.9632 0.9647549999999999 7.304750000000061e-07


In [16]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample5 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[5])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample5.append(count/10000)
    
print(max(Acc_results_G_sample5),min(Acc_results_G_sample5),np.mean(Acc_results_G_sample5),np.var(Acc_results_G_sample5))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.9288
SVD: 0.923
NetG_fixD, Acc at sample 1 0.9515


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.9263
SVD: 0.9209
NetG_fixD, Acc at sample 2 0.9552


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9274
SVD: 0.9189
NetG_fixD, Acc at sample 18 0.9579


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.9198
SVD: 0.9155
NetG_fixD, Acc at sample 19 0.9553


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.9275
SVD: 0.9218
NetG_fixD, Acc at sample 20 0.9554
0.9579 0.9497 0.95401 4.948900000000001e-06


In [17]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample6 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[6])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample6.append(count/10000)
    
print(max(Acc_results_G_sample6),min(Acc_results_G_sample6),np.mean(Acc_results_G_sample6),np.var(Acc_results_G_sample6))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.9233
SVD: 0.9155
NetG_fixD, Acc at sample 1 0.7867


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.9281
SVD: 0.9226
NetG_fixD, Acc at sample 2 0.8341


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9279
SVD: 0.9232
NetG_fixD, Acc at sample 18 0.7698


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.9268
SVD: 0.9221
NetG_fixD, Acc at sample 19 0.8387


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.9192
SVD: 0.9121
NetG_fixD, Acc at sample 20 0.8934
0.905 0.7698 0.84529 0.0016834348999999998


In [18]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample7 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[7])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample7.append(count/10000)
    
print(max(Acc_results_G_sample7),min(Acc_results_G_sample7),np.mean(Acc_results_G_sample7),np.var(Acc_results_G_sample7))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.9243
SVD: 0.9178999999999999
NetG_fixD, Acc at sample 1 0.5393


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.9213
SVD: 0.9168000000000001
NetG_fixD, Acc at sample 2 0.4054


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9227
SVD: 0.9162
NetG_fixD, Acc at sample 18 0.4504


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.9254
SVD: 0.9207
NetG_fixD, Acc at sample 19 0.405


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.9189
SVD: 0.9148000000000001
NetG_fixD, Acc at sample 20 0.441
0.5393 0.2811 0.4264400000000001 0.004701414399999999


In [19]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample8 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[8])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample8.append(count/10000)
    
print(max(Acc_results_G_sample8),min(Acc_results_G_sample8),np.mean(Acc_results_G_sample8),np.var(Acc_results_G_sample8))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.9256
SVD: 0.9252
NetG_fixD, Acc at sample 1 0.1223


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.931
SVD: 0.925
NetG_fixD, Acc at sample 2 0.129


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9298
SVD: 0.924
NetG_fixD, Acc at sample 18 0.1352


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.9211
SVD: 0.9162
NetG_fixD, Acc at sample 19 0.1423


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.9316
SVD: 0.9238999999999999
NetG_fixD, Acc at sample 20 0.142
0.1618 0.1 0.131195 0.00021991747499999995


In [20]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample9 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[9])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample9.append(count/10000)
    
print(max(Acc_results_G_sample9),min(Acc_results_G_sample9),np.mean(Acc_results_G_sample9),np.var(Acc_results_G_sample9))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.9223
SVD: 0.9218
NetG_fixD, Acc at sample 1 0.1068


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.64s/it]


----------------------------
Train_z, Test_z
PCA: 0.9306
SVD: 0.9289000000000001
NetG_fixD, Acc at sample 2 0.1285


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9344
SVD: 0.9308
NetG_fixD, Acc at sample 18 0.0892


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.9311
SVD: 0.9249
NetG_fixD, Acc at sample 19 0.1059


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


----------------------------
Train_z, Test_z
PCA: 0.9278
SVD: 0.9251
NetG_fixD, Acc at sample 20 0.1175
0.1379 0.0776 0.10802 0.00024915660000000004


In [21]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample10 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[10])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample10.append(count/10000)
    
print(max(Acc_results_G_sample10),min(Acc_results_G_sample10),np.mean(Acc_results_G_sample10),np.var(Acc_results_G_sample10))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


----------------------------
Train_z, Test_z
PCA: 0.9352
SVD: 0.9329
NetG_fixD, Acc at sample 1 0.0907


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.9299
SVD: 0.9264
NetG_fixD, Acc at sample 2 0.117


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9257
SVD: 0.9238999999999999
NetG_fixD, Acc at sample 18 0.1033


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.62s/it]


----------------------------
Train_z, Test_z
PCA: 0.9311
SVD: 0.9247
NetG_fixD, Acc at sample 19 0.0873


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.9319999999999999
SVD: 0.9293
NetG_fixD, Acc at sample 20 0.1111
0.1417 0.0788 0.10469999999999999 0.000223693
